Import libraries

In [ ]:
import os
import cv2
import math
import numpy
import pandas
import geopandas
import matplotlib.pyplot
import matplotlib.patches
import pysal
import pathlib
from PIL import Image
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import distance
from esda import Moran, Geary
from libpysal.weights import Queen
import ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual
from skimage import restoration

Input parameters

In [ ]:
number=0

In [ ]:
#warnings.filterwarnings('ignore')
BadLabel="_ORG"
impath="path"
labels=["dye1","dye2"]
labelR="BF"
imlabel=".tif"
invertBF=False
ImageSaveFolder="path-IMG"
csvSaveFolder="path-CSV"
if os.path.exists(ImageSaveFolder)==False:
    os.makedirs(ImageSaveFolder)
if os.path.exists(csvSaveFolder)==False:
    os.makedirs(csvSaveFolder)
pixeltoum = 0.178 #micrometros
AreaperField = 228.86*170.95*(10**-6) #area de cada campo en mm^2
squaredPixel = 1038*1388 #pixels
thr_value = 20
thr_value2 = 250
thr2_value=250
thr2_value2=250
intersection_threshold = 0.5  # Ajusta este valor según tu necesidad
x=0
showing=True
pixeltoum = 0.178 #micrometros
minArea=(1**2)*math.pi
maxArea=math.pi*50**2
cheackarea=False
Grain=False

Step by step

In [ ]:
showing=True

In [ ]:
## Posision de archivos al inicio
fn=0
label=labels[0]
imlist=[]
imlist2=[]
for (root,dirs,files) in os.walk(impath, topdown=True):
    if showing==True:
        print(str(x)+" "+root)
        print("")
    for file in files:
        if not file.endswith(imlabel):
            continue
        if label in file:
            if showing==True:                print("image file: "+file)
            imlist += [os.path.join(root,file)]
        if labelR in file:
            if showing==True: print("image file 2: "+file)
            imlist2 += [os.path.join(root,file)]

In [ ]:
Conds=[]
for Folder in [x for x in os.listdir(impath) if os.path.isdir(os.path.join(impath,x))]:
    Conds += [Folder]
print(Conds)

First condition con=0

In [ ]:
con=number
Cond=Conds[con]
print(Cond)

In [ ]:
imlistCon=[f for f in imlist if Cond in f]
imlist2Con=[f for f in imlist2 if Cond in f]
#thr2_value = thr2_values[con]
print(len(imlistCon))

For each file fn=0

In [ ]:
#fn=0
print(imlistCon[fn])
imn=imlistCon[fn]
print(imlist2Con[fn])
imn2=imlist2Con[fn]

In [ ]:
#check if file was already processed
SaveCSV=csvSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
pathlib.Path(SaveCSV.replace(SaveCSV.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
    if showing == True:
        print("Already processed")

In [ ]:
#open image 1 and 2
im=matplotlib.pyplot.imread(imn)
im2=matplotlib.pyplot.imread(imn2)
if showing==True:
    matplotlib.pyplot.imshow(im)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/1"+labels[0]+"-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
    matplotlib.pyplot.imshow(im2)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/1Bright-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()

In [ ]:
#Process image 1

#thr_value = 20
#thr_value2 = 250

imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) #threshold
if showing==True:
    matplotlib.pyplot.imshow(imgray)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/2DAPI-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
####
background = restoration.rolling_ball(imgray, radius=50)
if showing==True:
    matplotlib.pyplot.imshow(background)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/3DAPI-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
imgray2=imgray-background
if showing==True:
    matplotlib.pyplot.imshow(imgray2)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/4DAPI-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
###
ret, thresh = cv2.threshold(imgray2, thr_value,thr_value2,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)
if showing==True:
    matplotlib.pyplot.imshow(thresh)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/5DAPI-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()

In [ ]:

#Process image 2
imgray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY) #threshold
if showing==True:
    matplotlib.pyplot.imshow(imgray2)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/2Bright-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
'''
equ = cv2.equalizeHist(imgray2)
print("equ")
matplotlib.pyplot.imshow(equ)
matplotlib.pyplot.show()
ret,th = cv2.threshold(equ, 50, 255, 0)
print("thresh")
matplotlib.pyplot.imshow(th)
matplotlib.pyplot.show()
print("kernel")
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
matplotlib.pyplot.imshow(kernel)
matplotlib.pyplot.show()
print("dilate")
dilate = cv2.morphologyEx(th, cv2.MORPH_DILATE, kernel,3)
dilate = cv2.morphologyEx(dilate, cv2.MORPH_DILATE, kernel,3)
dilate = cv2.morphologyEx(dilate, cv2.MORPH_DILATE, kernel,3)
matplotlib.pyplot.imshow(dilate)
matplotlib.pyplot.show()
'''
###
#thr2_value=100
ret, thresh2 = cv2.threshold(imgray2, thr2_value,thr2_value2,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)
if invertBF:
    thresh2=cv2.bitwise_not(thresh2)
if showing==True:
    matplotlib.pyplot.imshow(thresh2)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/3Bright-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
thresh2


In [ ]:
thresh.size

In [ ]:
# obten los contornos desde las imagenes binarias

In [ ]:
#imagen 1 contornos
imc=matplotlib.pyplot.imread(imn)
if showing==True:
    matplotlib.pyplot.imshow(imc)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/6DAPI-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
if showing==True: print("getting contours")
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
imc=cv2.drawContours(imc, contours, -1, (0,255,0), 3)
if showing==True:
    matplotlib.pyplot.imshow(imc)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/6DAPI-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()

In [ ]:
#imagen 2 contornos (agujeros)
im2c=matplotlib.pyplot.imread(imn2)
if showing==True:
    matplotlib.pyplot.imshow(im2c)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/4Bright-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
contours2,_ = cv2.findContours(thresh2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
im2c=cv2.drawContours(im2c, contours2, -1, (0,255,0), 3)
if showing==True:
    matplotlib.pyplot.imshow(im2c)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/5Bright-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
    
#ve el area de los agujeros
colors, counts = numpy.unique(thresh2.reshape(-1, 1),return_counts = True,axis = 0)
for numb in range(len(colors)):
    color=colors[numb]
    count=counts[numb]
    #print(str(color)+": "+str(count))
sum(counts)
holeArea=counts[0]/thresh2.size


In [ ]:
#ahora filtra los contornos de la imagen 1 si tienen una tamaño menor que el area minimo o mayor que el area maxima o si se sobreponen con contornos de la imagen 2
filtered_contours = []
trash_contours=[]
cheackarea=True
#Iteramos sobre los contornos en contours
#for contour1 in range(len(contours)):
for cnt1 in range(len(contours)):
    #print("---------------------------------------------------------------------------------")
    #print(cnt1)
    contour1=contours[cnt1]
    keep_contour1 = True  # Variable para indicar si se debe mantener el contorno 1
    if cheackarea==True:
        area = cv2.contourArea(contour1)
        #print(str(maxArea)+" > "+str(area)+" > "+str(minArea))
        if maxArea < area or area < minArea:
            #print("Wont keep this contour")
            keep_contour1 = False
            trash_contours.append(contour1)
            continue
    '''
    if keep_contour1==True:
        print("this contour will be kept")
    '''
    #print("Keep processing ---------------------------")
    #for contour2 in contours2:
    for cnt2 in range(len(contours2)):
        #print(cnt2)
        contour2=contours2[cnt2]
        #make sure the area isnt too small
        area2 = cv2.contourArea(contour2)
        if area2 < math.pi*(2**2):#if the area is smaller than the minimun, continue with the next contour
            #print("the area is too small to matter")
            continue
        # Encuentra la intersección entre los dos contornos
        intersection = cv2.intersectConvexConvex(contour1, contour2)
        #print("checking the intersection")
        #print(intersection)
        # Verifica si la intersección es válida
        if intersection[0] == (0.0):
            #print("no intersection")
            continue
        elif intersection[0] != (0.0):
            if showing==True: print("the holes touch the colony, get rid of it")
            keep_contour1 = False
            trash_contours.append(contour1)
            #print("Break")
            break
    #print("still in the first loop")
    #save the contour
    if keep_contour1==True:
        filtered_contours.append(contour1)
    #muestra imagenes
if showing == True:
    print("this countours wont be keep")
    imc=matplotlib.pyplot.imread(imn)
    imc=cv2.drawContours(imc, trash_contours, -1, (0,255,0), 3)
    matplotlib.pyplot.imshow(imc, interpolation='nearest')
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/7DAPI-trash-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()
    # filtered_contours ahora contiene los contornos de la imagen 1 después de eliminar las superposiciones
    print("this countours will be keep")
    imc1=matplotlib.pyplot.imread(imn)
    imc1=cv2.drawContours(imc1, filtered_contours, -1, (0,255,0), 3)
    matplotlib.pyplot.imshow(imc1, interpolation='nearest')
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/8DAPI-keep-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    matplotlib.pyplot.show()

In [ ]:
NNs=[]
Positions,Majors,Minors,Angles,Areas,Perimeters,Circularities,AspectRats,Rounds=[],[],[],[],[],[],[],[],[]

for each contour ncnt1=0

In [ ]:
ncnt=0
contour1=filtered_contours[ncnt]

In [ ]:
distances=[]
for ncnt2 in range(len(filtered_contours)):
    (xc,yc),(major, minor),angle = (0,0),(0,0),0
    (xc2,yc2),(major2, minor2),angle2 = (0,0),(0,0),0
    if ncnt == ncnt2:
        continue
    if showing == True:
        image1=matplotlib.pyplot.imread(imlist[fn])
        matplotlib.pyplot.imshow(image1)
        print("image with two countours")
        image1=cv2.drawContours(image1, (filtered_contours[ncnt],filtered_contours[ncnt2]), -1, (0,255,0), 3)
        matplotlib.pyplot.imshow(image1, interpolation='nearest')
        matplotlib.pyplot.show()
    cnt = filtered_contours[ncnt]
    if 5 > len(filtered_contours[ncnt]):
        #print("not enough countours to get an ellipse in the first colony")
        circle = cv2.minEnclosingCircle(cnt)
        (xc,yc),rad = circle
    else:
        ellipse = cv2.fitEllipse(cnt)
        (xc,yc),(major, minor),angle = ellipse
    cnt2 = filtered_contours[ncnt2]
    if 5 > len(filtered_contours[ncnt2]):
        #print("not enough countours to get an ellipse in the second colony")
        circle=cv2.minEnclosingCircle(cnt2)
        (xc2,yc2),rad = circle
    else:
        ellipse = cv2.fitEllipse(cnt2)
        (xc2,yc2),(major2, minor2),angle2 = ellipse
    if showing == True:
        print("Position "+str(ncnt)+" :"+str((xc,yc))+" Postion "+str(ncnt2)+" :"+str((xc2,yc2)))
    #now check the distance between the two points, pitagoras
    distan = pixeltoum*math.sqrt((xc-xc2)**2+(yc-yc2)**2)
    if showing == True:
        print("the distance between this points is: "+str(distan)+" pixels or "+str(distan*pixeltoum))
    distances.append(distan)
if showing == True:
    print("The nearest neighbor is at: "+str(min(distances))+" μm")
try:
    NNs.append(min(distances))
except ValueError:
    NNs.append(None)

In [ ]:
#obten otros parametros importantes del contorno
circl=False
(xc,yc),(major, minor),angle = (0,0),(0,0),0
cnt = filtered_contours[ncnt]
try:
    ellipse = cv2.fitEllipse(cnt)
    (xc,yc),(major, minor),angle = ellipse
except  Exception as e:
    if showing ==True:
        print(e)
        print("not enough countours to get an ellipse in the first colony")
    circle = cv2.minEnclosingCircle(cnt)
    (xc,yc),major = circle
    minor=major
Area = cv2.contourArea(cnt)  # Area of first contour
Perimeter = cv2.arcLength(cnt, True)  # Perimeter of first contour 
major=major*pixeltoum
minor=minor*pixeltoum
Area=Area*pixeltoum*pixeltoum*1000
Perimeter=Perimeter*pixeltoum
#otros parametros
try:
    Ciruclarity = 4*(math.pi)*(Area/(Perimeter**2))
except  Exception as e:
    if showing==True: print(e)
    Ciruclarity=None
AspectRatio = major/minor
try:
    Roundness = 4*(Area/(((math.pi)*major)**2))
except:
    Roundness = None
try:
    xc,yc=int(xc),int(yc)
except  Exception as e:
    if showing == True: print(e)
if showing == True:
    print("Position:"+str([xc,yc]))
    print("Major: "+str(major)+" μm")
    print("Minor: "+str(minor)+" μm")
    print("Angle: "+str(angle)+" μm")
    print("Area: "+str(Area)+" nm^2")
    print("Perimeter: "+str(Perimeter)+" μm")
    print("Circularity: "+str(Ciruclarity))
    print("Aspect Ratio: "+str(AspectRatio))
    print("Roundness: "+str(Roundness))
Positions.append([xc,yc])
Majors.append(major)
Minors.append(minor)
Angles.append(angle)
Areas.append(Area)
Perimeters.append(Perimeter)
Circularities.append(Ciruclarity)
AspectRats.append(AspectRatio)
Rounds.append(Roundness)

Termina la imagen y guarda el dataframe

In [ ]:
PercentHole=[]
for num in range(len(Positions)):
    PercentHole.append(holeArea)
'''
print(len(Positions))
print(len(Majors))
print(len(Minors))
print(len(Angles))
print(len(Areas))
print(len(Perimeters))
print(len(Circularities))
print(len(AspectRats))
print(len(Rounds))
print(len(NNs))
print(len(PercentHole))
'''

In [ ]:
data=pandas.DataFrame({"Position": Positions,"Major [μm]": Majors,"Minor [μm]": Minors,
                       "Angle [μm]": Angles,"Area [nm^2]": Areas,"Perimeter [μm]": Perimeters,
                       "Circularity": Circularities,"Aspect Ratio": AspectRats,"Roundness": Rounds,
                       "NearestNeighbor [μm]": NNs,"Hole Area [%]": PercentHole})

In [ ]:
if showing==True: print("data is going to be saved")
data.to_csv(SaveCSV.split(".")[0]+".csv", index=False)

Ahora, abre todos los archivos de condiciones y haz un resumen con una linea por imagen

In [ ]:
#revisa todos los archivos csv obtenidos hasta ahora
con=7
Cond=Conds[con]
print(Cond)
imlistCon=[f for f in imlist if Cond in f]
imlist2Con=[f for f in imlist2 if Cond in f]
thr2_value = thr2_values[con]
print(len(imlistCon))

In [ ]:
#check if file was already processed
proc=0
noproc=0
for fn in range(len(imlistCon)):
    SaveCSV=csvSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
    pathlib.Path(SaveCSV.replace(SaveCSV.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
    if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
        proc=proc+1
    else:
        noproc=noproc+1
if showing == True:
    print(str(proc)+"/"+str(len(imlistCon)))
#f proc==len(imlistCon):
    #continuar

In [ ]:
AvMajor,AvMinor,AvAngle,AvArea,AvPer,AvCir,AvAR,AvRo,AvNN=[],[],[],[],[],[],[],[],[]
PercentHoles=[]
FieldArea=[]
RipleyKs,MoranIs,GearyCs,MoranIIs,GearyCIs,MoranCorrSizes,MoranCorrSizesR=[],[],[],[],[],[],[]
imlistNames=[]

continua cargando cada archivo para calcular sus variables globales. fn=0

In [ ]:
fn=2
imn=imlistCon[fn]
imn2=imlist2Con[fn]

In [ ]:
imlistNames.append(imn.split(".")[0].split("/")[-1].split("\\")[-1])
imlistNames

In [ ]:
SaveCSV=csvSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
pathlib.Path(SaveCSV.replace(SaveCSV.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
    data = pandas.read_csv(SaveCSV.split(".")[0]+".csv")
'''
else:
    continue
'''
print(SaveCSV)
data

In [ ]:
r_max=600
x,y,radius,coordinates=[],[],[],[]
for n in range(len(data["Position"])):
    try:
        x.append(int(data["Position"][n].replace("(","").replace(")","").split(",")[0]))
        y.append(int(data["Position"][n].replace("(","").replace(")","").split(",")[1]))
        radius.append(int(data["Major [μm]"][n]))
        coor=[int(data["Position"][n].replace("(","").replace(")","").split(",")[0]),int(data["Position"][n].replace("(","").replace(")","").split(",")[1])]
        coordinates.append(coor)
    except:
        continue
coordinatesN = numpy.array(coordinates)

In [ ]:
#Voronoi
if len(coordinatesN) > 2:
    vor = Voronoi(coordinatesN)
    fig = voronoi_plot_2d(vor)
    fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',line_width=2, line_alpha=0.6, point_size=2)
    if ImageSaveFolder != "":
        matplotlib.pyplot.savefig(ImageSaveFolder+"/Voronoi-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
    if showing == True:
        matplotlib.pyplot.show()
    else: matplotlib.pyplot.close()

In [ ]:
#Segundo Ripley's K
if showing==True: print("ripleyK")
r_values = numpy.linspace(0, r_max, num=100)  # You can adjust the number of bins
n = len(coordinatesN)
k_values = numpy.zeros(len(r_values))
for i, r in enumerate(r_values):
    k_values[i] = numpy.sum([numpy.sum(numpy.logical_and(distance.cdist([p], coordinatesN) <= r, distance.cdist([p], coordinatesN) > 0)) - 1 for p in coordinatesN]) / n
matplotlib.pyplot.plot(r_values, k_values, marker='o')
matplotlib.pyplot.axline((0, 0), slope=1, color='red', linestyle='--', label='$K(r) = r$')  # Add the reference line
matplotlib.pyplot.xlabel('Distance')
matplotlib.pyplot.ylabel('Ripley\'s K-function')
matplotlib.pyplot.legend()
if ImageSaveFolder != "":
    matplotlib.pyplot.savefig(ImageSaveFolder+"/RipleyK-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
if showing == True:
    matplotlib.pyplot.show()
else: matplotlib.pyplot.close()
max_k_index = numpy.argmax(k_values)  # Index of maximum K-value
max_r = r_values[max_k_index]  # Distance at maximum K-value
max_k = k_values[max_k_index]  # Maximum K-value
if max_k < max_r:
    interpretation = "Clustering"  # K(r) < r suggests clustering
elif max_k > max_r:
    interpretation = "Regular/Uniform"  # K(r) > r suggests regular or uniform distribution
else:
    interpretation = "Random"  # K(r) = r suggests random distribution
if showing == True:    print("Ripley's K:", interpretation)
RipleyKs.append(interpretation)

In [ ]:
# Create a GeoDataFrame from the colony data
geometry = geopandas.points_from_xy(x, y)
datag = {'radius': radius}  # Include other attributes if needed
gdf = geopandas.GeoDataFrame(datag, geometry=geometry)
# Create a spatial weights matrix (Queen contiguity)
w = Queen.from_dataframe(gdf)
if showing==True: print("moran and geary")
# Calculate Moran's I
moran = Moran(gdf.radius, w)
# Calculate Geary's C
geary = Geary(gdf.radius, w)
moran_value, geary_value=moran.I, geary.C
moran_interpretation = None
geary_interpretation = None
# Interpret Moran's I
if moran_value > 0:
    moran_interpretation = "Spatial Clustering"
elif moran_value < 0:
    moran_interpretation = "Spatial Dispersion"
else:
    moran_interpretation = "Random Spatial Pattern"
# Interpret Geary's C
if geary_value < 0.5:
    geary_interpretation = "Spatial Clustering"
elif geary_value > 0.5:
    geary_interpretation = "Spatial Dispersion"
else:
    geary_interpretation = "Random Spatial Pattern"
if showing == True:
    # Print results
    print("Moran's I:", moran.I)
    print("Geary's C:", geary.C)
    print("Interpretation of Moran's I:", moran_interpretation)
    print("Interpretation of Geary's C:", geary_interpretation)
MoranIs.append(moran.I)
GearyCs.append(geary.C)
MoranIIs.append(moran_interpretation)
GearyCIs.append(geary_interpretation)
size_list=radius
position_list=coordinates
#size_autocorrelation = calculate_size_autocorrelation(size_list, position_list, w)
size_list, position_list, spatial_weights = size_list, position_list, w
moran = Moran(size_list, spatial_weights)
size_autocorrelation=moran.I
if showing == True:
    print("Spatial Autocorrelation of Colony Sizes (Moran's I):", size_autocorrelation)
MoranCorrSizes.append(size_autocorrelation)
moran_i_value =  size_autocorrelation  # Replace with your calculated Moran's I value
#interpretation = interpret_autocorrelation_result(moran_i_value)
if moran_i_value > 0:
    interpretation = "Positive spatial autocorrelation: Larger sizes tend to be near other larger sizes."
elif moran_i_value < 0:
    interpretation = "Negative spatial autocorrelation: Smaller sizes tend to be near larger sizes."
else:
    interpretation = "No significant spatial autocorrelation: Colony sizes are randomly distributed."
if showing == True:
    print("Interpretation of Spatial Autocorrelation Result:", interpretation)
MoranCorrSizesR.append(interpretation)

In [ ]:
RipleyKs,MoranIs,GearyCs,MoranIIs,GearyCIs,MoranCorrSizes,MoranCorrSizesR

In [ ]:
AvMajor.append(data["Major [μm]"].mean(axis=0))
AvMinor.append(data["Minor [μm]"].mean(axis=0))
AvAngle.append(data["Angle [μm]"].mean(axis=0))
AvArea.append(data["Area [nm^2]"].mean(axis=0))
AvPer.append(data["Perimeter [μm]"].mean(axis=0))
AvCir.append(data["Circularity"].mean(axis=0))
AvAR.append(data["Aspect Ratio"].mean(axis=0))
AvRo.append(data["Roundness"].mean(axis=0))
AvNN.append(data["NearestNeighbor [μm]"].mean(axis=0))
FieldArea.append((data['Area [nm^2]'].sum())/(AreaperField*10**12)*100)#AreaperField in mm^2, *100 para obtener un porcentaje
PercentHoles.append(PercentHole)

In [ ]:
data2=pandas.DataFrame({"File Name":imlistNames,
                       "Major mean [μm]": AvMajor,
                       "Minor mean [μm]": AvMinor,
                       "Angle mean [μm]": AvAngle,
                       "Colony area mean [nm^2]": AvArea,
                       "Perimeter mean [μm]": AvPer,
                       "Circularity mean": AvCir,
                       "Aspect ratio mean": AvAR,
                       "Roundness mean": AvRo,
                       "Nearest Neighbor mean [μm]": AvNN,
                       "Colonized area [%]": FieldArea,
                       "Ripley's K": RipleyKs,
                       "Moran's I": MoranIs,
                       "Geary's C": GearyCs,
                       "Moran's I's interpretation": MoranIIs,
                       "Geary's C interpretation": GearyCIs,
                       "Radius correlation with position by Moran's I": MoranCorrSizes,
                       "Radius correlation with position by Moran's I interpretation": MoranCorrSizesR,
                       "Hole percent":PercentHoles})
if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
    data3=pandas.read_csv(SaveCSV.split(".")[0]+".csv")
    data3 = pandas.concat([data3, data2], ignore_index=True, sort=False)
    data3.to_csv(SaveCSV.split(".")[0]+".csv", index=False)
else:
    data2.to_csv(SaveCSV.split(".")[0]+".csv", index=False)

In [ ]:
data2